# ETL SIMPLES: Bronze -> Silver

## Objetivo:
1. **EXTRACT** - Carregar dados brutos (Bronze)
2. **TRANSFORM** - Limpar e tratar dados (outliers, tipos, nulos)
3. **LOAD** - Popular banco PostgreSQL

Confirme objetivos e a sequência das etapas antes de executar.
---

## 1. Imports e Configuração

Carregar bibliotecas, definir caminhos e validar dependências.

In [9]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas carregadas!")
print(f"Execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Bibliotecas carregadas!
Execução: 2026-01-12 13:00:51


In [10]:
import re

def converter_para_snake_case(nome_coluna):
    """Converte nomes de colunas para snake_case."""
    nome = nome_coluna.strip()
    nome = re.sub(r'[\s\-]+', '_', nome)
    nome = re.sub(r'[^a-z0-9_]', '', nome.lower())
    nome = re.sub(r'_+', '_', nome)
    return nome.strip('_')

def limpar_texto(texto):
    """Remove espaços extras, quebras de linha e caracteres de controle."""
    if pd.isna(texto):
        return texto
    texto = str(texto).strip()
    texto = re.sub(r'[\r\n\t]+', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto

def converter_preco(valor):
    """Converte string de preço para float. Ex: '$1,234.56' → 1234.56"""
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip()
    if valor_str == '':
        return np.nan
    valor_str = re.sub(r'[\$\s]', '', valor_str)
    valor_str = valor_str.replace(',', '')
    try:
        return float(valor_str)
    except ValueError:
        return np.nan

def converter_percentual(valor):
    """Converte percentual string para float (0-1). Ex: '95%' → 0.95"""
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip()
    if valor_str == '' or valor_str == '%':
        return np.nan
    valor_str = valor_str.replace('%', '').strip()
    try:
        return float(valor_str) / 100
    except ValueError:
        return np.nan

def converter_booleano(valor):
    """Converte valores t/f, true/false, yes/no para boolean."""
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip().lower()
    if valor_str in ['t', 'true', 'yes', 'y', '1']:
        return True
    elif valor_str in ['f', 'false', 'no', 'n', '0']:
        return False
    else:
        return np.nan

def remover_linhas_totalmente_vazias(df):
    """Remove linhas onde todas as colunas são NaN."""
    return df.dropna(how='all').reset_index(drop=True)

print("Funções utilitárias carregadas!")

Funções utilitárias carregadas!


## Funções Utilitárias

Revisar e testar funções utilitárias (conversões/limpeza) com amostras.

## 2. EXTRACT - Carregar Dados Bronze

Ler arquivos CSV brutos do diretório raw e validar contagens/tipos.

In [11]:
PATH_RAW = '../raw/'

print("Carregando dados Bronze...\n")

df_listings = pd.read_csv(PATH_RAW + 'dados_brutos_listings.csv', low_memory=False)
df_calendar = pd.read_csv(PATH_RAW + 'dados_brutos_calendar.csv')
df_reviews = pd.read_csv(PATH_RAW + 'dados_brutos_reviews.csv')

print(f"Listings: {len(df_listings):,} registros")
print(f"Calendar: {len(df_calendar):,} registros")
print(f"Reviews: {len(df_reviews):,} registros")

Carregando dados Bronze...

Listings: 5,764 registros
Calendar: 1,048,575 registros
Reviews: 62,976 registros


## 3. TRANSFORM - Limpar e Tratar Dados

Aplicar conversões, imputação e remoção de outliers; verificar amostras antes/depois.

### 3.1. Limpar LISTINGS

In [12]:
print("Limpando LISTINGS...")

df_listings['price_clean'] = df_listings['price'].str.replace('$', '').str.replace(',', '', regex=False)
df_listings['price_clean'] = pd.to_numeric(df_listings['price_clean'], errors='coerce')

for col in ['security_deposit', 'cleaning_fee', 'extra_people']:
    if col in df_listings.columns:
        df_listings[f'{col}_clean'] = df_listings[col].str.replace('$', '').str.replace(',', '', regex=False)
        df_listings[f'{col}_clean'] = pd.to_numeric(df_listings[f'{col}_clean'], errors='coerce')

for col in ['bathrooms', 'bedrooms', 'beds']:
    if col in df_listings.columns:
        df_listings[col] = pd.to_numeric(df_listings[col], errors='coerce')
        df_listings[col] = df_listings[col].fillna(0).round().astype(int)

Q1 = df_listings['price_clean'].quantile(0.25)
Q3 = df_listings['price_clean'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

print(f"   • Outliers - Limites: ${lower:.2f} até ${upper:.2f}")

df_listings = df_listings[
    (df_listings['price_clean'] >= lower) & 
    (df_listings['price_clean'] <= upper) &
    (df_listings['price_clean'] > 0)
]

print(f"   • Após remoção outliers: {len(df_listings):,} listings")

for col in ['host_since', 'first_review', 'last_review']:
    if col in df_listings.columns:
        df_listings[col] = pd.to_datetime(df_listings[col], errors='coerce')

bool_cols = ['host_is_superhost', 'instant_bookable', 'is_location_exact', 
             'require_guest_profile_picture', 'require_guest_phone_verification',
             'host_has_profile_pic', 'host_identity_verified']
for col in bool_cols:
    if col in df_listings.columns:
        df_listings[col] = df_listings[col] == 't'

colunas_selecionadas = [
    'id', 'name', 'property_type', 'room_type', 'bed_type',
    'accommodates', 'bathrooms', 'bedrooms', 'beds',
    'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'market', 
    'country_code', 'country', 'latitude', 'longitude', 'is_location_exact',
    'price_clean', 'security_deposit_clean', 'cleaning_fee_clean', 
    'guests_included', 'extra_people_clean',
    'minimum_nights', 'maximum_nights', 'instant_bookable', 'cancellation_policy',
    'require_guest_profile_picture', 'require_guest_phone_verification',
    'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'number_of_reviews', 'first_review', 'last_review', 'reviews_per_month',
    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'amenities',
    'host_id', 'host_name', 'host_since', 'host_location', 'host_response_time',
    'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
    'host_neighbourhood', 'host_listings_count', 'host_total_listings_count',
    'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
    'calculated_host_listings_count'
]

colunas_existentes = [col for col in colunas_selecionadas if col in df_listings.columns]
df_listings_clean = df_listings[colunas_existentes].copy()

renomear = {
    'id': 'listing_id',
    'name': 'listing_name',
    'price_clean': 'listing_price',
    'security_deposit_clean': 'security_deposit',
    'cleaning_fee_clean': 'cleaning_fee',
    'extra_people_clean': 'extra_people'
}
df_listings_clean.rename(columns=renomear, inplace=True)

print(f"Listings limpos: {len(df_listings_clean):,} registros")
print(f"Colunas selecionadas: {len(df_listings_clean.columns)}")

Limpando LISTINGS...
   • Outliers - Limites: $-247.50 até $668.50
   • Após remoção outliers: 5,244 listings
Listings limpos: 5,244 registros
Colunas selecionadas: 61


### 3.2. Limpar CALENDAR

Converter datas, limpar preços e padronizar campo de disponibilidade.

In [13]:
print("Limpando CALENDAR...")

df_calendar['date'] = pd.to_datetime(df_calendar['date'])

df_calendar['price_clean'] = df_calendar['price'].str.replace('$', '').str.replace(',', '')
df_calendar['price_clean'] = pd.to_numeric(df_calendar['price_clean'], errors='coerce')

df_calendar['available'] = df_calendar['available'] == 't'

df_calendar_clean = df_calendar[['listing_id', 'date', 'available', 'price_clean']].copy()
df_calendar_clean.columns = ['listing_id', 'calendar_date', 'calendar_available', 'calendar_price']

print(f"Calendar limpo: {len(df_calendar_clean):,} registros")

Limpando CALENDAR...
Calendar limpo: 1,048,575 registros


### 3.3. Limpar REVIEWS

Converter datas, remover reviews inválidos e manter últimas avaliações relevantes.

In [14]:
print("Limpando REVIEWS...")

df_reviews['date'] = pd.to_datetime(df_reviews['date'])

df_reviews_clean = df_reviews[['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name']].copy()
df_reviews_clean.columns = ['listing_id', 'review_id', 'review_date', 'reviewer_id', 'reviewer_name']

df_reviews_clean = df_reviews_clean.dropna(subset=['review_id', 'listing_id'])

print(f"Reviews limpos: {len(df_reviews_clean):,} registros")

Limpando REVIEWS...
Reviews limpos: 62,976 registros


## 4. CONSOLIDATE - Juntar Tudo (One Big Table)

Juntar calendar (90d) e reviews (últimas 10) com listings e revisar nulos/duplicados.

In [15]:
print("Consolidando dados...")

# Otimização: Pegar últimos 90 dias de calendar por listing (reduz de 1M para ~470k)
print("Filtrando últimos 90 dias de calendar por listing...")
df_calendar_90d = df_calendar_clean.sort_values('calendar_date', ascending=False).groupby('listing_id').head(90)
print(f"   • Calendar reduzido: {len(df_calendar_90d):,} registros")

# Otimização: Pegar últimas 10 reviews por listing
df_reviews_10 = df_reviews_clean.sort_values('review_date', ascending=False).groupby('listing_id').head(10)
print(f"   • Reviews reduzido: {len(df_reviews_10):,} registros")

df_silver = df_listings_clean.merge(
    df_calendar_90d,
    on='listing_id',
    how='left'
)

df_silver = df_silver.merge(
    df_reviews_10,
    on='listing_id',
    how='left'
)

print(f"\nDados consolidados: {len(df_silver):,} registros")
print(f"Colunas: {len(df_silver.columns)}")

print("\nTratando valores nulos...")

df_silver = df_silver.where(pd.notnull(df_silver), None)

nulos = df_silver.isnull().sum()
print(f"   • Colunas com nulos: {(nulos > 0).sum()}")
print(f"   • Total de nulos: {nulos.sum():,}")

Consolidando dados...
Filtrando últimos 90 dias de calendar por listing...
   • Calendar reduzido: 258,570 registros
   • Reviews reduzido: 22,367 registros

Dados consolidados: 1,033,112 registros
Colunas: 68

Tratando valores nulos...
   • Colunas com nulos: 30
   • Total de nulos: 2,544,453


## 5. LOAD - Popular Banco PostgreSQL

Testar conexão e inserção com pequena amostra antes do carregamento completo.

In [16]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'austin_airbnb',
    'user': 'postgres',
    'password': 'postgres'
}

engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

print("Conectando ao PostgreSQL...")
print(f"   Host: {DB_CONFIG['host']}")
print(f"   Database: {DB_CONFIG['database']}")

try:
    with engine.connect() as conn:
        print("   Conexão OK!")
except Exception as e:
    print(f"   ERRO na conexão: {e}")
    raise

print(f"\nTotal de registros consolidados: {len(df_silver):,}")
print("Carregando TODOS os registros (pode demorar 10-20 minutos)")

df_to_load = df_silver

print(f"\nPopulando {len(df_to_load):,} registros na tabela silver.one_big_table...")
print("Aguarde... NÃO INTERROMPA!")

df_to_load.to_sql(
    name='one_big_table',
    con=engine,
    schema='silver',
    if_exists='replace',
    index=False,
    chunksize=5000,
    method='multi'
)

print(f"\n{len(df_to_load):,} registros inseridos com sucesso!")
print(f"Tabela: silver.one_big_table")
print(f"Colunas: {len(df_to_load.columns)}")

Conectando ao PostgreSQL...
   Host: localhost
   Database: austin_airbnb
   Conexão OK!

Total de registros consolidados: 1,033,112
Carregando TODOS os registros (pode demorar 10-20 minutos)

Populando 1,033,112 registros na tabela silver.one_big_table...
Aguarde... NÃO INTERROMPA!

1,033,112 registros inseridos com sucesso!
Tabela: silver.one_big_table
Colunas: 68


## 6. VALIDAÇÃO - Verificar no Banco

Executar contagens, estatísticas e amostras para validar a carga no banco.

In [17]:
print("Validando dados no banco...\n")

query_count = "SELECT COUNT(*) as total FROM silver.one_big_table"
df_count = pd.read_sql(query_count, engine)
print(f"Registros no banco: {df_count['total'][0]:,}")

query_sample = "SELECT * FROM silver.one_big_table LIMIT 5"
df_sample = pd.read_sql(query_sample, engine)
print(f"\nAmostra (5 registros):")
print(df_sample[['listing_id', 'listing_name', 'listing_price', 'neighbourhood_cleansed']].to_string(index=False))

query_stats = """
SELECT 
    COUNT(DISTINCT listing_id) as total_listings,
    AVG(listing_price) as preco_medio,
    MIN(listing_price) as preco_min,
    MAX(listing_price) as preco_max
FROM silver.one_big_table
WHERE listing_price IS NOT NULL
"""
df_stats = pd.read_sql(query_stats, engine)
print(f"\nEstatísticas:")
print(f"   Listings únicos: {df_stats['total_listings'][0]:,}")
print(f"   Preço médio: ${df_stats['preco_medio'][0]:.2f}")
print(f"   Preço mínimo: ${df_stats['preco_min'][0]:.2f}")
print(f"   Preço máximo: ${df_stats['preco_max'][0]:.2f}")

print("\n" + "="*60)
print("ETL FINALIZADO COM SUCESSO!")
print("BANCO POPULADO: silver.one_big_table")
print("="*60)

Validando dados no banco...

Registros no banco: 1,033,112

Amostra (5 registros):
 listing_id                  listing_name  listing_price  neighbourhood_cleansed
      72635 3 Private Bedrooms, SW Austin          300.0                   78739
      72635 3 Private Bedrooms, SW Austin          300.0                   78739
      72635 3 Private Bedrooms, SW Austin          300.0                   78739
      72635 3 Private Bedrooms, SW Austin          300.0                   78739
      72635 3 Private Bedrooms, SW Austin          300.0                   78739

Estatísticas:
   Listings únicos: 5,244
   Preço médio: $159.64
   Preço mínimo: $14.00
   Preço máximo: $650.00

ETL FINALIZADO COM SUCESSO!
BANCO POPULADO: silver.one_big_table


In [18]:
df_silver

,listing_id,listing_name,property_type,room_type,bed_type,accommodates,bathrooms,bedrooms,beds,neighbourhood_cleansed,...,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calendar_date,calendar_available,calendar_price,review_id,review_date,reviewer_id,reviewer_name
0,72635,"3 Private Bedrooms, SW Austin",House,Private room,Real Bed,6,2,1,3,78739,...,True,False,1,2016-11-05,True,300.0,205626.0,2011-03-21,421148.0,Luca
1,72635,"3 Private Bedrooms, SW Austin",House,Private room,Real Bed,6,2,1,3,78739,...,True,False,1,2016-11-04,True,300.0,205626.0,2011-03-21,421148.0,Luca
2,72635,"3 Private Bedrooms, SW Austin",House,Private room,Real Bed,6,2,1,3,78739,...,True,False,1,2016-11-03,True,300.0,205626.0,2011-03-21,421148.0,Luca
3,72635,"3 Private Bedrooms, SW Austin",House,Private room,Real Bed,6,2,1,3,78739,...,True,False,1,2016-11-02,True,300.0,205626.0,2011-03-21,421148.0,Luca
4,72635,"3 Private Bedrooms, SW Austin",House,Private room,Real Bed,6,2,1,3,78739,...,True,False,1,2016-11-01,True,300.0,205626.0,2011-03-21,421148.0,Luca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033107,6063670,Austin's Downtown Garden Suite,Apartment,Entire home/apt,Real Bed,4,1,1,2,78701,...,True,True,11,NaT,None,NaN,30909036.0,2015-04-28,3879870.0,Chris
1033108,8422925,Two beds in Downtown Austin!,Condominium,Private room,Real Bed,2,1,1,2,78701,...,True,True,2,NaT,None,NaN,NaN,NaT,NaN,None
1033109,3345881,Casa Romántica en Picos de Europa,House,Entire home/apt,Real Bed,2,1,0,1,78733,...,True,False,1,NaT,None,NaN,32625767.0,2015-05-19,29226090.0,Sofia
1033110,8954997,Living room with bed,Apartment,Shared room,Real Bed,1,1,1,1,78728,...,True,True,1,NaT,None,NaN,NaN,NaT,NaN,None
